In [67]:
"""
logistic regression
maximum entropy IIS
"""

'\nlogistic regression\nmaximum entropy IIS\n'

In [68]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [69]:
X_train =np.array([[0.1,1.1],[6.8,7.1],[-3.5,-4.1],[2.0,2.7],[4.1,2.8],[3.1,5.0],[-0.8,-1.3],[0.9,1.2],[5.0,6.4],[3.9,4.0],
                  [7.1,4.2],[-1.4,-4.3],[4.5,0.0],[6.3,1.6],[4.2,1.9],[1.4,-3.2],[2.4,-4.0],[2.5,-6.1],[8.4,3.7],[4.1,-2.2]],
                 dtype='float')
y_train =np.array([1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0],dtype='float')
print(X_train.shape)
print(y_train.shape)

(20, 2)
(20,)


In [70]:
y_train =y_train[:,np.newaxis]
print(y_train.shape)
data =np.concatenate([X_train,y_train],axis=1)
print(data[1,:-1].shape)

(20, 1)
(2,)


In [71]:
def sigmoid(x):
    return 1.0/(1.0+(np.exp(-x)))
def crossentropy(t,y):
    [n,p]=t.shape
    loss =0;
    for i in range(n):
        loss= loss +t[i]*(np.log(y[i]+1e-7))+(1-t[i])*(np.log(1-y[i]+1e-7))
    return -1*loss

class LogisticRegression(object):
    '''
    f(y|x) =1/(1+exp(-1*alpha))
    '''
    def __init__(self,beta=None):
        self.beta =beta
    def train(self,data,method='grad'):
        '''
        method: grad gradient descent
                iris iteratiove reweighted least squares
                dfp
                bfgs
        '''
        [n,p]=data.shape
        p =p-1
        X =data[:,0:p].copy()
        X =np.concatenate([X,np.ones((n,1),dtype=float)],axis=1)
        t =data[:,-1].copy()
        t =t[:,np.newaxis]
        y =np.zeros((n,1),dtype=float)
        beta =0.5*np.ones((p+1,1),dtype=float)
        beta_old =np.ones((p+1,1),dtype=float)
        hessian =np.zeros((p+1,p+1),dtype=float)
        eps =0.001
        if(method=='irls'):
            while(np.linalg.norm((beta_old -beta))>eps and np.linalg.norm(beta)>eps):
                
                beta_old =beta
                y =np.array(list(map(sigmoid,[x.dot(beta) for x in X ])),dtype=float)
                RR =np.multiply(y,1-y)+1e-9   #numerical stability
                R =np.diag((RR.flatten()))
                Z =X@beta -np.linalg.inv(R)@(y-t)
                hessian =X.T@R@X
                beta =np.linalg.inv(hessian)@(X.T@R)@Z
            self.beta =beta
            return beta
        elif(method=='grad'):
            while(np.linalg.norm((beta_old-beta))>eps and  np.linalg.norm(beta)>eps):
                beta_old =beta
                y=np.array(list(map(sigmoid,[x.dot(beta) for x in X])),dtype=float)
                grad =X.T@(y-t)
                beta =beta -grad
            self.beta=beta
            return beta
        elif(method=='dfp'):
            G =np.eye(p+1)
            grad_old =np.ones((p+1,1),dtype=float)
            grad =np.ones((p+1,1),dtype=float)
            while(np.linalg.norm(beta_old -beta)>eps  and np.linalg.norm(beta)>eps):
                #print('old ',beta_old,'\n','current ',beta)
                #print('grad ',grad)
                grad_old =grad
                beta_old =beta
                y =np.array(list(map(sigmoid,[x.dot(beta) for x in X])),dtype=float)
                p =-G@grad_old
                
                
                """
                #0.618 linear search  
                # single valley interval
                
                low =0
                high=0.1
                gamma =1.1
                low_test =0
                high_test =0
                
                low_val =crossentropy(t,y)
                y =np.array(list(map(sigmoid,[x.dot(beta+high*p) for x in X])),dtype=float)
                high_val =crossentropy(t,y)
                while(high_val<=low_val):
                    high =high*gamma
                    y =np.array(list(map(sigmoid,[x.dot(beta+high*p) for x in X])),dtype=float)
                    high_val =crossentropy(t,y)
                    print('seeking valley ',' low_val ',low_val,' high_val ',high_val)
                    
               
                c =0.5*(np.sqrt(5)-1)
                print(' low ',low,' high ',high)
                print('top: low_val ',low_val,'high_val ',high_val)
                low_test= low + (high-low)*(1.0-c)
                high_test= low + (high-low)*c
                low_prime =low
                high_prime =high
                #search for minnimal
                while(((high-low)/(high_prime-low_prime))>=0.001):
                    y_low =np.array(list(map(sigmoid,[x.dot(beta+low_test*p) for x in X])),dtype=float)
                    y_high =np.array(list(map(sigmoid,[x.dot(beta+high_test*p) for x in X])),dtype=float)
                    low_val =crossentropy(t,y_low)
                    high_val =crossentropy(t,y_high)
                    print('in while: low_val ',low_val,'high_val ',high_val)
                    if(low_val<high_val):
                        high =high_test
                        low_test =low +(1.0-c)*(high-low)
                        high_test =low +(high-low)*c
                    else:
                        low =low_test
                        low_test =low+(high-low)*(1.0 - c)
                        high_test =low +(high-low)*c
                print('p ',p)
                print('$\lambda$ ',0.5*(low+high))
                beta =beta+0.5*(low+high)*p
                
                """
                
                alpha =0.25
                f =0.8
                step =1
                prime_val =crossentropy(t,y)
                y =np.array(list(map(sigmoid,[x.dot(beta+step*p) for x in X])),dtype=float)
                search_val =crossentropy(t,y)
                while(search_val>(prime_val+alpha*step*(grad.T@p))):
                    step =f*step
                    y =np.array(list(map(sigmoid,[x.dot(beta+step*p) for x in X])),dtype=float)
                    search_val =crossentropy(t,y)    
                beta =beta +step*p
                
    
    
                y =np.array(list(map(sigmoid,[x.dot(beta) for x in X])),dtype=float)
                grad =X.T@(y-t)
                #theta_k =0.5*(low+high)*p
                theta_k =step*p
                y_k =grad -grad_old
                P_k =(theta_k@theta_k.T)/(theta_k.T@y_k)
                Q_k =-1*(G@y_k@y_k.T@G)/(y_k.T@G@y_k)
                G =G+P_k+Q_k
            self.beta=beta
            return beta
        elif(method=='bfgs'):
            B =np.eye(p+1)
            grad_old =np.ones((p+1,1),dtype=float)
            grad =np.ones((p+1,1),dtype=float)
            while(np.linalg.norm(beta_old -beta)>eps  and np.linalg.norm(beta)>eps):
                #print('old ',beta_old,'\n','current ',beta)
                #print('grad ',grad)
                grad_old =grad
                beta_old =beta
                y =np.array(list(map(sigmoid,[x.dot(beta) for x in X])),dtype=float)
                p =-np.linalg.inv(B)@grad_old
                
                
                """
                #0.618 linear search  
                # single valley interval
                
                low =0
                high=0.1
                gamma =1.1
                low_test =0
                high_test =0
                
                low_val =crossentropy(t,y)
                y =np.array(list(map(sigmoid,[x.dot(beta+high*p) for x in X])),dtype=float)
                high_val =crossentropy(t,y)
                while(high_val<=low_val):
                    high =high*gamma
                    y =np.array(list(map(sigmoid,[x.dot(beta+high*p) for x in X])),dtype=float)
                    high_val =crossentropy(t,y)
                    print('seeking valley ',' low_val ',low_val,' high_val ',high_val)
                    
               
                c =0.5*(np.sqrt(5)-1)
                print(' low ',low,' high ',high)
                print('top: low_val ',low_val,'high_val ',high_val)
                low_test= low + (high-low)*(1.0-c)
                high_test= low + (high-low)*c
                low_prime =low
                high_prime =high
                #search for minnimal
                while(((high-low)/(high_prime-low_prime))>=0.001):
                    y_low =np.array(list(map(sigmoid,[x.dot(beta+low_test*p) for x in X])),dtype=float)
                    y_high =np.array(list(map(sigmoid,[x.dot(beta+high_test*p) for x in X])),dtype=float)
                    low_val =crossentropy(t,y_low)
                    high_val =crossentropy(t,y_high)
                    print('in while: low_val ',low_val,'high_val ',high_val)
                    if(low_val<high_val):
                        high =high_test
                        low_test =low +(1.0-c)*(high-low)
                        high_test =low +(high-low)*c
                    else:
                        low =low_test
                        low_test =low+(high-low)*(1.0 - c)
                        high_test =low +(high-low)*c
                print('p ',p)
                print('$\lambda$ ',0.5*(low+high))
                beta =beta+0.5*(low+high)*p
                
                """
                
                alpha =0.25
                f =0.8
                step =1
                prime_val =crossentropy(t,y)
                y =np.array(list(map(sigmoid,[x.dot(beta+step*p) for x in X])),dtype=float)
                search_val =crossentropy(t,y)
                while(search_val>(prime_val+alpha*step*(grad.T@p))):
                    step =f*step
                    y =np.array(list(map(sigmoid,[x.dot(beta+step*p) for x in X])),dtype=float)
                    search_val =crossentropy(t,y)    
                beta =beta +step*p
                
    
    
                y =np.array(list(map(sigmoid,[x.dot(beta) for x in X])),dtype=float)
                grad =X.T@(y-t)
                #theta_k =0.5*(low+high)*p
                theta_k =step*p
                y_k =grad -grad_old
                P_k =(y_k@theta_k.T)/(theta_k.T@theta_k)
                Q_k =-1*(B@theta_k@theta_k.T@B)/(theta_k.T@B@theta_k)
                B =B+P_k+Q_k
            self.beta =beta
            return beta
        
        def predict(self,data):
            X=data.copy()

            [n,p]=X.shape
            X =np.concatenate([X,np.ones((n,1),dtype=float)],axis=1)
            p=p+1
            y_predict =np.array(list(map(sigmoid,[x.dot(self.beta) for x in X])),dtype=float)
            return y_predict
            
                
                
                
                
                
                
                
            

            
        
            
        
            
        
            
        

        

In [72]:
lr =LogisticRegression()
beta =lr.train(data,method='bfgs')
X =data[:,:-1].copy()
t =data[:,-1].copy()
X =np.concatenate([X,np.ones((20,1))],axis=1)
t =t[:,np.newaxis]
y =np.array(list(map(sigmoid,[x.dot(beta) for x in X])),dtype=float)



In [73]:
print('t ',t)
print('y ',y)

t  [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
y  [[1.00000000e+000]
 [1.00000000e+000]
 [1.00000000e+000]
 [1.00000000e+000]
 [1.00000000e+000]
 [1.00000000e+000]
 [1.00000000e+000]
 [1.00000000e+000]
 [1.00000000e+000]
 [1.00000000e+000]
 [3.64134374e-033]
 [1.44087337e-028]
 [2.58992559e-078]
 [4.17301028e-085]
 [3.86909375e-014]
 [1.51472519e-079]
 [1.84485467e-132]
 [1.58421262e-196]
 [3.05232012e-086]
 [1.80355137e-130]]


In [74]:


print('grad ',lr.train(data,method='grad'))
print('irls ',lr.train(data,method='irls'))
print('dfp ',lr.train(data,method='irls'))
print('bfgs ',lr.train(data,method='bfgs'))

grad  [[-24.53806799]
 [ 28.13204894]
 [ 38.11747126]]
irls  [[-2.41849180e+08]
 [ 2.09407138e+08]
 [ 4.16091170e+08]]
dfp  [[-2.41849180e+08]
 [ 2.09407138e+08]
 [ 4.16091170e+08]]
bfgs  [[-68.24230896]
 [ 66.99691812]
 [128.4403822 ]]
